# Mic check

In [16]:
import sounddevice as sd


def get_default_input_device_info():
    default_input_device = sd.default.device[0]  # get the ID of the default input device
    device_info = sd.query_devices(default_input_device)
    print(device_info)

# Get the info of the default input device
get_default_input_device_info()

{'name': 'MacBook Air 마이크', 'index': 3, 'hostapi': 0, 'max_input_channels': 1, 'max_output_channels': 0, 'default_low_input_latency': 0.0336875, 'default_low_output_latency': 0.01, 'default_high_input_latency': 0.043020833333333335, 'default_high_output_latency': 0.1, 'default_samplerate': 48000.0}


In [17]:
import sounddevice as sd

def list_input_devices():
    devices = sd.query_devices()
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:  # this is an input device
            print(f"Device #{i} name: {device['name']}")

# List available input devices (including microphones)
list_input_devices()

Device #1 name: 갤럭시 S2 마이크
Device #2 name: USB PnP Sound Device
Device #3 name: MacBook Air 마이크


In [18]:
import numpy as np

# Choose the device to use for recording
device_id = 2  # replace with the ID of the device you want to use
duration = 3  # seconds

# Create a buffer to store the audio data
buffer = np.zeros((duration * 44100,))
buffer_index = 0

# Define a callback function to process the audio input
def audio_callback(indata, frames, time, status):
    global buffer_index
    volume_norm = np.linalg.norm(indata) * 10
    print(f'\r{"|" * int(volume_norm)}', end='')  # print a simple "volume bar"

    # Store the incoming data in the buffer
    buffer[buffer_index:buffer_index+frames] = indata[:, 0]
    buffer_index += frames

# Create a stream object
stream = sd.InputStream(callback=audio_callback, device=device_id, channels=1, samplerate=44100)

# Start the stream
with stream:
    # Record for 3 seconds
    sd.sleep(duration * 1000)

# Play back the recorded sound
sd.play(buffer, samplerate=44100)

### Import models


In [19]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import resnet18

def load_model_interactive(model_dir="models"):
    # List all models in the directory
    model_files = [f for f in os.listdir(model_dir) if f.endswith('.pth') and not f.startswith('._')]
    
    # Display the models to the user
    for idx, model_name in enumerate(model_files, 1):
        print(f"{idx}. {model_name}")
    
    # Get user input
    selected_idx = int(input("Enter the number corresponding to the model you wish to load: ")) - 1
    model_path = os.path.join(model_dir, model_files[selected_idx])
    model = None   
    try:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            
            # Load the resnet18 model structure
            model = resnet18(pretrained=False)
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(512, 14)  # Assuming selected_labels is globally accessible
            
            # Load the state dict
            state_dict = torch.load(model_path, map_location=device)
            new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
            model.load_state_dict(new_state_dict)
            
            print(f"Model loaded from {model_path}")
            model = model.to(device)
            model = model.eval()
    except Exception as e:
        print(f"Failed to load the model. Error: {e}")

    return model

# For demonstration purposes, the function will display model names but won't actually load them here.
# Please run this in your local environment for actual model loading.
model = load_model_interactive()

1. Model2_005.pth
2. Model2_006.pth
3. Model2_007.pth
4. Model2_008.pth
5. Model2_009.pth
6. Model2_010.pth
7. Batch4-1k_001.pth
8. Batch4-1k_002.pth
9. Batch4-1k_003.pth
10. Batch4-1k_004.pth
11. Model2_001.pth
12. Model2_002.pth
13. Model2_003.pth
14. Model2_004.pth
Model loaded from models/Batch4-1k_004.pth


In [20]:
class MinMaxNormalize(nn.Module):
    def __init__(self, min_val=None, max_val=None):
        super(MinMaxNormalize, self).__init__()
        self.min_val = min_val
        self.max_val = max_val

    def forward(self, tensor):
        if self.min_val is None or self.max_val is None:
            min_val = torch.min(tensor)
            max_val = torch.max(tensor)
        else:
            min_val = self.min_val
            max_val = self.max_val
        
        normalized_tensor = (tensor - min_val) / (max_val - min_val)
        return normalized_tensor

In [21]:
class MonoToColor(nn.Module):
    def __init__(self, num_channels=3):
        super(MonoToColor, self).__init__()
        self.num_channels = num_channels

    def forward(self, tensor):
        return tensor.repeat(self.num_channels, 1, 1)

In [22]:
from torchvision import transforms
import torchaudio

SAMPLE_RATE = 22050

# Apply the same transformation as used during training
transformation = transforms.Compose([
    torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE, n_mels=40),# higher the better but more complex. For talking we use 128, for sound effect, about 40.
    torchaudio.transforms.AmplitudeToDB(stype='power', top_db=80),
    MinMaxNormalize(),
    MonoToColor()
])

In [46]:
import torch.nn.functional as F

def continuous_sound_prediction(model, device, transformation, sample_rate, device_id):
    labels = [
        "nothing a", "Car Horn", "Bell Ring", "Dog Bark", "nothing b", 
        "nothing c", "Glass Shatter", "Nothing d", "Nothing e", "Door Nock", 
        "Nothing f", "Nothing g", "Siren", "Nothing h"
    ]

    for count in range(101):
        try:
            # Recording
            duration = 2.0  # seconds
            recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, device=device_id)
            sd.wait()
            
            # Preprocessing
            recording = torch.from_numpy(recording).float().transpose(0, 1)
            if recording.shape[0] > 1:
                recording = torch.mean(recording, dim=0, keepdim=True)
            recording = nn.functional.pad(recording, (0, sample_rate - recording.shape[1]))
            
            # Transformation
            recording = transformation(recording)
            
            # Prediction
            model.eval()
            with torch.no_grad():
                recording = recording.to(device)
                outputs = model(recording.unsqueeze(0))
                #probabilities = F.softmax(outputs, dim=1)
                probabilities = torch.sigmoid(outputs)

                _, predicted = outputs.max(1)
            # Print results
            probs = [f"{label} {prob:.2%}" for label, prob in zip(labels, probabilities[0])]
            #print(f"{count} / {' / '.join(probs)}")



            max_prob, predicted_label_idx = probabilities[0].max(0)
            max_prob_label = labels[predicted_label_idx]
            print(f"{count} / Predicted class: {max_prob_label} / Max Probability: {max_prob:.2%}")


        except Exception as e:
            print(f"Error during prediction: {e}")
            break

    print("Finished continuous sound prediction.")
device = 'cpu'
continuous_sound_prediction(model, device, transformation, SAMPLE_RATE, device_id)

0 / Predicted class: Nothing e / Max Probability: 88.65%
1 / Predicted class: Nothing h / Max Probability: 51.20%
2 / Predicted class: Nothing d / Max Probability: 55.97%
3 / Predicted class: Nothing e / Max Probability: 72.38%
4 / Predicted class: Nothing d / Max Probability: 80.13%
5 / Predicted class: Nothing e / Max Probability: 80.37%
6 / Predicted class: Nothing h / Max Probability: 91.28%
7 / Predicted class: Nothing h / Max Probability: 90.40%
8 / Predicted class: Dog Bark / Max Probability: 75.20%
9 / Predicted class: Nothing e / Max Probability: 85.84%
10 / Predicted class: Nothing h / Max Probability: 27.63%
11 / Predicted class: Nothing h / Max Probability: 73.96%
12 / Predicted class: Bell Ring / Max Probability: 95.46%
13 / Predicted class: Door Nock / Max Probability: 82.67%
14 / Predicted class: Nothing d / Max Probability: 29.41%
15 / Predicted class: Dog Bark / Max Probability: 40.52%
16 / Predicted class: Door Nock / Max Probability: 70.89%
17 / Predicted class: Dog 

KeyboardInterrupt: 

In [ ]:
import pyaudio
import wave
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

# Modified record_audio function
def record_audio(input_device_index, sample_rate=44100, channels=1, duration=2):
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=sample_rate,
                    input=True,
                    input_device_index=input_device_index,
                    frames_per_buffer=1024)

    frames = []

    for _ in range(0, int(sample_rate / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    # Convert frames to numpy array
    byte_string = b''.join(frames)
    audio_array = np.frombuffer(byte_string, dtype=np.int16)
    return audio_array


# Additional transformation methods
def _right_pad_if_necessary(signal, target_sample_rate):
    length_signal = signal.shape[1]
    if length_signal < target_sample_rate:
        num_missing_samples = target_sample_rate - length_signal
        last_dim_padding = (0, num_missing_samples)
        signal = nn.functional.pad(signal, last_dim_padding)
    return signal

def _resample_if_necessary(signal, sr, target_sample_rate):
    if sr != target_sample_rate:
        resampler = torchaudio.transforms.Resample(sr, target_sample_rate)
        signal = resampler(signal)
    return signal

def _mix_down_if_necessary(signal):
    if signal.shape[0] > 1:
        signal = torch.mean(signal, dim=0, keepdim=True)
    return signal


# MonoToColor and MinMaxNormalize classes
class MonoToColor(nn.Module):
    def __init__(self, num_channels=3):
        super(MonoToColor, self).__init__()
        self.num_channels = num_channels

    def forward(self, tensor):
        return tensor.repeat(self.num_channels, 1, 1)

class MinMaxNormalize(nn.Module):
    def __init__(self, min_val=None, max_val=None):
        super(MinMaxNormalize, self).__init__()
        self.min_val = min_val
        self.max_val = max_val

    def forward(self, tensor):
        if self.min_val is None or self.max_val is None:
            min_val = torch.min(tensor)
            max_val = torch.max(tensor)
        else:
            min_val = self.min_val
            max_val = self.max_val
        
        normalized_tensor = (tensor - min_val) / (max_val - min_val)
        return normalized_tensor


# Apply the same transformation as used during training
SAMPLE_RATE = 22050
transformation = transforms.Compose([
    torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE, n_mels=40),
    torchaudio.transforms.AmplitudeToDB(stype='power', top_db=80),
    MinMaxNormalize(),
    MonoToColor()
])


# Modify continuous_sound_prediction function to use record_audio
def continuous_sound_prediction(model, device, transformation, sample_rate, device_id):
    labels = [
        "children", "nothing2", "drilling", "engine", "siren", 
        "gunshot", "aircon", "jackhammer", "carhorn", "glass", 
        "nock", "street_music", "dog_bark", "nothing1"
    ]

    for count in range(101):
        try:
            # Recording
            audio_data = record_audio(device_id, sample_rate=sample_rate)
            
            # Preprocessing
            audio_tensor = torch.from_numpy(audio_data).float().unsqueeze(0)
            audio_tensor = _right_pad_if_necessary(audio_tensor, SAMPLE_RATE)
            audio_tensor = _mix_down_if_necessary(audio_tensor)
            
            # Transformation
            transformed_audio = transformation(audio_tensor)
            
            # Prediction
            model.eval()
            with torch.no_grad():
                transformed_audio = transformed_audio.to(device)
                outputs = model(transformed_audio.unsqueeze(0))
                probabilities = F.softmax(outputs, dim=1)
                _, predicted = outputs.max(1)

            # Print results
            probs = [f"{label} {prob:.2%}" for label, prob in zip(labels, probabilities[0])]
            print(f"{count} / {' / '.join(probs)}")

        except Exception as e:
            print(f"Error during prediction: {e}")
            break

    print("Finished continuous sound prediction.")

device = 'cpu'
continuous_sound_prediction(model, device, transformation, SAMPLE_RATE, device_id)

||PaMacCore (AUHAL)|| Error on line 2523: err='-50', msg=Unknown Error


KeyboardInterrupt: 